# Segmenting and Clustering Neighborhoods in Toronto

### In this notebook I will be scraping data from the wikipedia page https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M for the Neighborhoods of Toronto.
Lets start by imported the needed libraries.

In [4]:
import pandas as pd
import numpy as np

Fortunately we can just use pandas for the scraping. We don't need other libraries.

In [5]:
df=pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


We need to fix the columns. Postcode should be Postalcode and Neighbourhood should be Neighborhood.

In [6]:
df.columns =["Postalcode","Borough","Neighborhood"] #correct the 1st column name
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


We then need to drop all the rows where Borough is 'Not assigned'

In [7]:
df=df[df['Borough']!='Not assigned']
df.reset_index(drop=True, inplace=True)
df.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


If there is a Neighborhood that is 'Not assigned' we have to change the Neighborhoods value to the Boroughs value.

In [8]:
df[df['Neighborhood']=='Not assigned'] #lets see which neighborhood is not assigned

,Postalcode,Borough,Neighborhood
6,M9A,Queen's Park,Not assigned


In [9]:
df['Neighborhood'] =  np.where(df['Neighborhood'] == 'Not assigned', df['Borough'], df['Neighborhood']) #if neighborhood is not assigned replace with borough

In [10]:
df[df['Postalcode']=='M9A'] #check if it worked

,Postalcode,Borough,Neighborhood
6,M9A,Queen's Park,Queen's Park


More than one neighborhood can exist in one postal code area. So we have a lot of rows that have the same Postalcode but different Neighboorhoods. We are going to combine these rows into 1 row with the neighborhoods separated with a comma.

In [11]:
df = df.groupby(['Postalcode','Borough'])['Neighborhood'].apply(','.join).reset_index() #group by postalcode
df.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [12]:
df.shape

(103, 3)

In [13]:
!wget -q -O 'geospatial_data.csv' https://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [14]:
geospatial = pd.read_csv('geospatial_data.csv')
geospatial.columns =["Postalcode","Latitude","Longitude"]
geospatial.head()

,Postalcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
neighborhoods = pd.merge(df, geospatial, on='Postalcode')

In [17]:
neighborhoods.head(12)

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848
